In [1]:
import os
import redis
import pickle
import numpy as np
import keras
import redisConfig
from kerasPredictCMD import get_outputs_cmd

# start redis server and config the maximum memory available
redisConfig.run()

Using TensorFlow backend.


Configured


In [2]:
# model = keras.models.load_model('alexnet-cifar10_origin.h5')
# new_output = keras.layers.Softmax()(keras.layers.Dense(10)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model
# model.summary()

# model = keras.applications.ResNet50()
# new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model
# model.summary()

# model = keras.applications.MobileNetV2()
# new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model
# model.summary()

model = keras.applications.VGG16()
new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
new_model = keras.models.Model(input=model.input, output=new_output)
new_model.layers[-2].set_weights(model.layers[-1].get_weights())
model = new_model

/data/yylaiai/anaconda3/envs/audee_test/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("so...)`


In [3]:
failed_case_dir = 'saved_failed/vgg-16/8'

model.load_weights(os.path.join(failed_case_dir, 'weight'))
x = np.load(os.path.join(failed_case_dir, 'input.npy'))

In [4]:
r = redis.Redis(db=1)
r.flushdb()

# upload test inputs to redis db
r.hset('model', 0, pickle.dumps(model))
r.hset('input', 0, pickle.dumps(x))

1

In [5]:
cmd1 = get_outputs_cmd('tensorflow', 1, 0, 0, 0, 'all')
cmd2 = get_outputs_cmd('theano', 1, 0, 0, 0, 'all')

os.system(cmd1)
os.system(cmd2)

0

In [6]:
p1 = pickle.loads(r.hget('predictions_0', 'tensorflow'))
p2 = pickle.loads(r.hget('predictions_0', 'theano'))

In [7]:
print("Layers with NaN:\n")
print('{:>5} {:>12} {:>12}'.format('layer', 'tensorflow', 'theano'))
for i in range(len(model.layers)):
    print('{:>5} {:>12} {:>12}'.format(i, str(np.isnan(p1[i]).any()), str(np.isnan(p2[i]).any())))

Layers with NaN:

layer   tensorflow       theano
    0        False        False
    1        False        False
    2        False        False
    3        False        False
    4        False        False
    5        False        False
    6        False        False
    7        False        False
    8        False        False
    9        False        False
   10        False        False
   11        False        False
   12        False        False
   13         True         True
   14        False         True
   15         True         True
   16         True         True
   17         True         True
   18        False         True
   19        False         True
   20         True         True
   21         True         True
   22         True         True
   23         True         True
